In [ ]:
import pandas as pd
import os

def procesar_archivos_excel(base_path, meses, hojas_interes):
    for mes in meses:
        file_path = os.path.join(base_path, mes, f'DATOS EN FORMATO REUTILIZABLE {mes.upper()}.xlsx')
        if not os.path.exists(file_path):
            print(f"El archivo {file_path} no existe.")
            continue
        
        # Crear un diccionario para almacenar los DataFrames con el nombre de cada hoja
        dfs = {}
        
        # Leer cada hoja de interés y almacenarla en el diccionario
        for hoja in hojas_interes:
            try:
                df = pd.read_excel(file_path, sheet_name=hoja, header=2)  # Usar header=2 para que tome los títulos correctos
                dfs[hoja] = df
            except Exception as e:
                print(f"Error al leer la hoja {hoja} del archivo {file_path}: {e}")
        
        # Guardar cada DataFrame en un archivo CSV
        for hoja, df in dfs.items():
            output_path = os.path.join(base_path, mes, f"{hoja.lower().replace(' ', '_')}.csv")
            df.to_csv(output_path, index=False)
            print(f"Archivo guardado: {output_path}")

# Ruta base y meses a procesar
base_path = 'C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/'
meses = [
    'Abril 2024', 'Marzo 2024', 'Febrero 2024', 'Enero 2024',
    'Diciembre 2023', 'Noviembre 2023', 'Octubre 2023',
    'Septiembre 2023', 'Agosto 2023', 'Julio 2023',
    'Junio 2023', 'Mayo 2023', 'Abril 2023', 'Marzo 2023',
    'Febrero 2023', 'Enero 2023'
]
hojas_interes = ['SEGURIDAD', 'PERS. DETENIDAS X DISTRITOS', 'ACCIDENTES', 'EXPEDIENTES OAC', 'VENTA AMBULANTE', 'AA.TT.']

# Llamar a la función
procesar_archivos_excel(base_path, meses, hojas_interes)

## UNIR DATASET INDICES DE CRIMINALIDAD.

In [ ]:
def unir_archivos_csv(base_path, meses, archivos_csv):
    dfs_meses = {}

    # Recorrer cada carpeta de mes y leer los archivos CSV
    for mes in meses:
        carpeta = os.path.join(base_path, mes)
        dfs = {}
        for archivo in archivos_csv:
            ruta_archivo = os.path.join(carpeta, archivo)
            if os.path.exists(ruta_archivo):
                df = pd.read_csv(ruta_archivo)
                if 'COMISARÍAS' in df.columns:
                    df.rename(columns={'COMISARÍAS': 'DISTRITOS'}, inplace=True)
                dfs[archivo] = df
            else:
                print(f"El archivo {ruta_archivo} no existe.")
        
        # Verificar que todos los DataFrames tienen la columna 'DISTRITOS'
        for archivo, df in dfs.items():
            if 'DISTRITOS' not in df.columns:
                print(f"El archivo {archivo} no contiene la columna 'DISTRITOS'. Las columnas son: {df.columns}")

        # Inicializar el DataFrame final con el primer DataFrame de la lista
        if archivos_csv[0] in dfs:
            df_ind_criminalidad = dfs[archivos_csv[0]]
        
            # Realizar un outer join con los demás DataFrames basado en la columna 'DISTRITOS'
            for archivo in archivos_csv[1:]:
                if archivo in dfs and 'DISTRITOS' in dfs[archivo].columns:
                    df_ind_criminalidad = pd.merge(df_ind_criminalidad, dfs[archivo], on='DISTRITOS', how='outer')
                else:
                    print(f"Omitiendo el archivo {archivo} porque no tiene la columna 'DISTRITOS'.")
            
            # Guardar el DataFrame resultante de cada mes en el diccionario
            dfs_meses[mes] = df_ind_criminalidad
        else:
            print(f"No se encontró el archivo {archivos_csv[0]} en el mes {mes}.")

    # Unir los DataFrames de todos los meses
    df_final = pd.concat(dfs_meses.values(), ignore_index=True)
    
    return df_final

# Definir la lista de archivos CSV
archivos_csv = [
    'aa.tt..csv', 'accidentes.csv', 'expedientes_oac.csv',
    'pers._detenidas_x_distritos.csv', 'seguridad.csv', 'venta_ambulante.csv'
]

# Llamar a la función y obtener el DataFrame final
df_final = unir_archivos_csv(base_path, meses, archivos_csv)

# Guardar el DataFrame final en un archivo CSV
df_final.to_csv(os.path.join(base_path, 'indice_criminalidad_completo.csv'), index=False)
print("Archivo final guardado: indice_criminalidad_completo.csv")

## EJECUTABLE.

In [1]:
# Convertir archivos Excel a CSV
procesar_archivos_excel(base_path, meses, hojas_interes)

# Unir archivos CSV en un único DataFrame
df_final = unir_archivos_csv(base_path, meses, archivos_csv)

# Guardar el DataFrame final en un archivo CSV
df_final.to_csv(os.path.join(base_path, 'indice_criminalidad_completo.csv'), index=False)
print("Archivo final guardado: indice_criminalidad_completo.csv")

NameError: name 'procesar_archivos_excel' is not defined